# Lecture 11: Logistic Regression and Classification

In classification problem, the response variables $y$ are discrete, representing different catagories. For simplicity, we will first introduce the **binary classification case** -- $y$ has only two categories, denoted as $0$ and $1$.

## Model-setup

**Assumption 1**: Dependent on the variable $x$, the response variable $y$ has different **probabilities** to take value in 0 or 1. Instead of predicting exact value of 0 or 1, we are actually predicting the **probabilities**.

**Assumption 2**: Logistic function assumption. Given $x$, what is the probability to observe $y=1$?

$$P(y=1|\mathbf{x})=f(\mathbf{x};\mathbf{\beta}) = \frac{1}{1 + \exp(-\tilde{x}\mathbf{\beta})}
=: \sigma(\tilde{x}\mathbf{\beta}). $$

where $\sigma(z)=\frac{1}{1+\exp{(-x)}}$ is called [standard logistic function](https://en.wikipedia.org/wiki/Logistic_regression#:~:text=Logistic%20regression%20is%20a%20statistical,a%20form%20of%20binary%20regression), or sigmoid function in deep learning. Recall that $\beta\in\mathbb{R}^{p+1}$ and $\tilde{x}$ is the "augmented" sample with first element one to incorporate intercept in the linear function.

**Equivalent expression**:
  - Denote $p = P(y=1|\mathbf{x})$, then we can write in linear form
  $$\ln\frac{p}{1-p}=\tilde{x}\beta$$
  - Since $y$ only takes value in 0 or 1, we have
  $$P(y|\mathbf{x},\beta) = f(\mathbf{x};\beta)^y \big(1 - f(\mathbf{x};\beta) \big)^{1-y}$$
  
**MLE (Maximum Likelihood Estimation)**

Assume the samples are independent. The overall probibility to witness the whole training dataset
$$
{\begin{aligned}
&P(\mathbf{y}\; | \; \mathbf{X};\beta )\\
=&\prod _{i=1}^N P\left(y^{(i)}\mid \mathbf{x}^{(i)};\beta\right)\\
=&\prod_{i=1}^N f\big(\mathbf{x}^{(i)};\beta \big)^{y^{(i)}} 
\Big(1-f\big(\mathbf{x}^{(i)};\beta\big) \Big)^{\big(1-y^{(i)}\big)}
\end{aligned}}.
$$

By maximizing the logarithm of likelihood function, then we derive the **loss function** to be minimized
$$
L (\beta) = L (\beta; X,\mathbf{y}) = - \frac{1}{N}\sum_{i=1}^N 
\Bigl\{y^{(i)} \ln\big( f(\mathbf{x}^{(i)};\beta) \big) 
+ (1 - y^{(i)}) \ln\big( 1 - f(\mathbf{x}^{(i)};\beta) \big) \Bigr\}.
$$

The loss function also has clear probabilistic interpretations. Given $i$-th sample, the vector of true labels $(y^{i},1-y^{i})$ can also be viewed as the probability distribution. Then the loss function is the mean of all [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) across samples, i.e. "distance" between true probability distribution and estimated probability distribution via logistic model.

## Algorithm

Take the gradient (left as exercise -- if you like)
$$
\frac{\partial L (\beta)}{\partial \beta_{k}} =\frac{1}{N}\sum_{i=1}^N  \big(\sigma(\tilde{x}^{(i)}\beta)  - y^{(i)} \big) \tilde{x}^{(i)}_k.
$$

In vector form
$$
\nabla_{\beta} \big( L (\beta) \big) = \sum_{i=1}^N  \big(\sigma(\tilde{x}^{(i)})  - y^{(i)} \big) \tilde{x}^{(i)} 
=\frac{1}{N}\sum_{i=1}^N \big( f(\mathbf{x}^{(i)};\beta)  - y^{(i)} \big) \tilde{x}^{(i)}.
$$

This is still the nonlinear function of $\beta$, indicating that we cannot derive something like "normal equations" in OLS. The solution here is [numerical optimization](https://github.com/Jaewan-Yun/optimizer-visualization).

The simplest algorithm in optimization is [gradient descent (GD)](https://en.wikipedia.org/wiki/Gradient_descent#:~:text=Gradient%20descent%20is%20a%20first,function%20at%20the%20current%20point.). $$\beta_{k+1}=\beta_{k}-\eta\nabla L(\beta).$$

Here the step size $\eta$ is also called **learning rate** in machine learning. Note that it is indeed the Euler's scheme to solve the ODE $$\dot{\beta} = -\nabla L(\beta).$$

By setting certain stopping criterion for GD, we think that we have approximated the optimized solution $\hat{\beta}$.

## Making predictions

Now with the estimated $\hat{\beta}$ and given a new data $x^{new}$, we calculate the probability that $y^{new}=1$ as $f(\mathbf{x};\mathbf{\beta})$. If is greater than 0.5, we assign that $y^{new}=1$. 

For the whole dataset, the **accuracy** is defined as ratio of number of correct predictions to the total number of samples.

## Example Code

In [13]:
import numpy as np

class myLogisticRegression():
    """ Logistic Regression classifier -- this only works for the binary case.
    Parameters:
    -----------
    learning_rate: float
        The step length that will be taken when following the negative gradient during
        training.
    """
    def __init__(self, learning_rate=.1):
        
        # learning rate can also be in the fit method
        self.learning_rate = learning_rate
        

    def fit(self, data, y, n_iterations = 1000):
        """ 
        don't forget the document string in methods
        """
        ones = np.ones((data.shape[0],1)) # column of ones 
        X = np.concatenate((ones, data), axis = 1) # the augmented matrix, \tilde{X} in our lecture
        eta = self.learning_rate
        
        beta  = np.zeros(np.shape(X)[1]) # initialize beta, can be other choices

        for k in range(n_iterations):
            dbeta = self.loss_gradient(beta,X,y) # write another function to compute gradient
            beta = beta - eta * dbeta # the formula of GD
            # this step is optional -- just for inspection purposes
            if k % 50 == 0: # plot weights and print loss every 50 steps
                print("loss after", k+1, "iterations is: ", self.loss(beta,X,y))
        
        self.coeff = beta
        
    def predict(self, data):
        ones = np.ones((data.shape[0],1)) # column of ones 
        X = np.concatenate((ones, data), axis = 1) # the augmented matrix, \tilde{X} in our lecture
        beta = self.coeff # the estimated beta
        y_pred = np.round(self.sigmoid(np.dot(X,beta))).astype(int) # >0.5: ->1 else,->0 -- note that we always use Numpy universal functions when possible
        return y_pred
    
    def score(self, data, y_true):
        ones = np.ones((data.shape[0],1)) # column of ones 
        X = np.concatenate((ones, data), axis = 1) # the augmented matrix, \tilde{X} in our lecture
        y_pred = self.predict(data)
        acc = np.mean(y_pred == y_true) # number of correct predictions/N
        return acc
    
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    def loss(self,beta,X,y):
        f_value = self.sigmoid(np.dot(X,beta))
        loss_value = np.log(f_value + 1e-10) * y + (1.0 - y)* np.log(1 - f_value + 1e-10) # avoid nan issues
        return -np.mean(loss_value)
                          
    def loss_gradient(self,beta,X,y):
        f_value = self.sigmoid(np.dot(X,beta))                  
        gradient_value = (f_value - y).reshape(-1,1)*X # this is the hardest expression -- check yourself
        return np.mean(gradient_value, axis=0)

In [14]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y = True)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [4]:
X_train.shape

(512, 30)

In [16]:
lg = myLogisticRegression(learning_rate=1e-5)
lg.fit(X_train,y_train,n_iterations = 20000) # what about increase n_iterations?

loss after 1 iterations is:  0.7704000919325609
loss after 51 iterations is:  0.9234485854021112
loss after 101 iterations is:  0.540964240285867
loss after 151 iterations is:  0.39124404681558
loss after 201 iterations is:  0.37036060679508653
loss after 251 iterations is:  0.3539655742179028
loss after 301 iterations is:  0.34040008047058745
loss after 351 iterations is:  0.3290043880434598
loss after 401 iterations is:  0.3193284051902458
loss after 451 iterations is:  0.311041748365799
loss after 501 iterations is:  0.3038878556607375
loss after 551 iterations is:  0.2976620578084048
loss after 601 iterations is:  0.29220035526617916
loss after 651 iterations is:  0.28737177960898685
loss after 701 iterations is:  0.283071941752563
loss after 751 iterations is:  0.2792174560306317
loss after 801 iterations is:  0.2757413356503129
loss after 851 iterations is:  0.2725893486410744
loss after 901 iterations is:  0.2697172067962873
loss after 951 iterations is:  0.26708842141622324
los

loss after 9951 iterations is:  0.21214983067129695
loss after 10001 iterations is:  0.21209051523044697
loss after 10051 iterations is:  0.21203146158651243
loss after 10101 iterations is:  0.21197266645399337
loss after 10151 iterations is:  0.2119141266219995
loss after 10201 iterations is:  0.21185583895215604
loss after 10251 iterations is:  0.21179780037657545
loss after 10301 iterations is:  0.21174000789589287
loss after 10351 iterations is:  0.2116824585773627
loss after 10401 iterations is:  0.21162514955301487
loss after 10451 iterations is:  0.21156807801786826
loss after 10501 iterations is:  0.21151124122819925
loss after 10551 iterations is:  0.21145463649986376
loss after 10601 iterations is:  0.21139826120667116
loss after 10651 iterations is:  0.2113421127788074
loss after 10701 iterations is:  0.21128618870130694
loss after 10751 iterations is:  0.2112304865125707
loss after 10801 iterations is:  0.21117500380292942
loss after 10851 iterations is:  0.21111973821325
l

loss after 17801 iterations is:  0.20492558073843936
loss after 17851 iterations is:  0.2048890186340453
loss after 17901 iterations is:  0.20485254619537968
loss after 17951 iterations is:  0.20481616306265932
loss after 18001 iterations is:  0.20477986887872715
loss after 18051 iterations is:  0.20474366328901544
loss after 18101 iterations is:  0.20470754594151064
loss after 18151 iterations is:  0.20467151648671822
loss after 18201 iterations is:  0.20463557457762882
loss after 18251 iterations is:  0.20459971986968434
loss after 18301 iterations is:  0.20456395202074557
loss after 18351 iterations is:  0.20452827069105972
loss after 18401 iterations is:  0.20449267554322906
loss after 18451 iterations is:  0.20445716624218
loss after 18501 iterations is:  0.20442174245513262
loss after 18551 iterations is:  0.20438640385157125
loss after 18601 iterations is:  0.20435115010321517
loss after 18651 iterations is:  0.2043159808839901
loss after 18701 iterations is:  0.2042808958700001

In [17]:
lg.score(X_test,y_test)

1.0

In [7]:
lg.score(X_train,y_train)

0.916015625

In [8]:
lg.predict(X_test)

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1])

In [9]:
y_test

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1])

In [10]:
lg.coeff

array([ 2.66882874e-03,  1.83927907e-02, -4.36239390e-03,  8.66487934e-02,
        1.49727981e-02,  5.50578751e-05, -6.71061493e-04, -1.14024608e-03,
       -4.51040831e-04,  1.06678514e-04,  8.62208844e-05,  1.72299429e-04,
        4.51211649e-04, -2.32558967e-03, -2.93966958e-02, -5.28628701e-06,
       -1.83325756e-04, -2.46370803e-04, -5.80574855e-05, -9.52561495e-06,
       -1.16457037e-05,  1.92488199e-02, -1.67105144e-02,  6.60427872e-02,
       -2.88220491e-02, -2.09664782e-05, -2.48568195e-03, -3.30713576e-03,
       -8.60537728e-04, -1.96407733e-04, -8.95680417e-05])

In [11]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

/Users/cliffzhou/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9824561403508771

In [12]:
clf.score(X_train,y_train)

0.953125

It's very normal that our result is different with sklearn. In sklearn [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), they use different algorithms to solve the optimization problem , and even the model is different (adding regularization terms!)